# Chat with Paper Agent

In this notebook, we'll build an interactive chat interface that can answer questions about a research paper. This is like having an expert assistant who has read the paper and can explain it to you!

## What You'll Learn
- How to use the OpenAI Conversations API for stateful, multi-turn chat
- How to give an AI agent context about a specific document
- How to build a Q&A system for research papers

## Use Case
Research papers can be dense and difficult to understand. With this chat agent, you can:
- Ask questions about specific sections
- Get explanations of complex concepts in simple terms
- Explore the paper interactively at your own pace

The Conversations API automatically maintains context, so the agent remembers what you've discussed!

## Setup

First, let's import our libraries and set up the OpenAI client.

In [ ]:
from openai import OpenAI
from pypdf import PdfReader
from IPython.display import Markdown, display

# Initialize the OpenAI client
client = OpenAI()

## Load the Research Paper

We'll extract the text from a PDF so our agent can reference it when answering questions.

In [ ]:
def load_pdf_text(file_path):
    """Loads text from a PDF file."""
    reader = PdfReader(file_path)
    text = "\n\n".join([page.extract_text() for page in reader.pages])
    return text

# Load the Word2Vec paper (a classic in NLP!)
paper_text = load_pdf_text("../assets/paper3.pdf")

print(f"Loaded paper with {len(paper_text)} characters")
print(f"Approximately {len(paper_text.split())} words")

## Create a Conversation

The Conversations API creates a "conversation" object that stores all the messages. This is perfect for chat interfaces because it automatically maintains context between turns.

In [ ]:
# Create a new conversation
conversation = client.conversations.create(
    metadata={"paper": "word2vec", "user_id": "tutorial_user"}
)

print(f"Created conversation with ID: {conversation.id}")
print(f"Metadata: {conversation.metadata}")

## Give the Agent Context

Before asking questions, we need to "teach" the agent about the paper. We do this by sending the paper text as the first message with clear instructions.

In [ ]:
# System instructions for the agent
instructions = """
You are a helpful research assistant who explains academic papers in clear, simple terms.
You have been given the full text of a research paper.
Answer questions about the paper accurately and concisely.
If you don't know something, say so - don't make up information.
"""

# Send the paper content to establish context
context_response = client.responses.create(
    model="gpt-5-mini",
    instructions=instructions,
    input=f"Here is the research paper you'll be answering questions about:\n\n{paper_text}\n\nPlease confirm you've read and understood the paper.",
    conversation=conversation.id
)

print("Agent's response:")
print(context_response.output_text)

## Ask Questions About the Paper

Now we can have a natural conversation! Each question automatically has access to:
1. The original paper text
2. All previous questions and answers

This is the power of the Conversations API - it handles context management for you.

In [ ]:
def ask_question(question):
    """
    Ask the agent a question about the paper.
    The conversation ID keeps track of context automatically.
    """
    response = client.responses.create(
        model="gpt-5-mini",
        input=question,
        conversation=conversation.id
    )
    return response.output_text

# Question 1: Get a simple summary
q1 = "What is the main idea of this paper in one sentence?"
a1 = ask_question(q1)

print(f"Q: {q1}")
print(f"A: {a1}")
print()

In [ ]:
# Question 2: Ask about methodology
q2 = "How does the Skip-gram model work?"
a2 = ask_question(q2)

print(f"Q: {q2}")
print(f"A: {a2}")
print()

In [ ]:
# Question 3: Follow-up question (tests context retention)
q3 = "How is that different from CBOW?"
a3 = ask_question(q3)

print(f"Q: {q3}")
print(f"A: {a3}")
print()

In [ ]:
# Question 4: Ask about results
q4 = "What were the key findings or results?"
a4 = ask_question(q4)

print(f"Q: {q4}")
print(f"A: {a4}")
print()

## View the Entire Conversation

The Conversations API keeps a complete history. Let's retrieve and display it!

In [ ]:
# Retrieve the conversation history
conversation_data = client.conversations.retrieve(conversation.id)

print(f"Conversation ID: {conversation_data.id}")
print(f"Created at: {conversation_data.created_at}")
print(f"Total messages: {len(conversation_data.metadata)}")

# Note: The actual message history is accessed through the responses
# Each response in our conversation is linked through the conversation_id

## Display the Full Q&A Session

Let's create a nicely formatted view of our chat session.

In [ ]:
# Create a formatted summary of the Q&A session
qa_summary = f"""
# Chat with Paper: Q&A Summary

## Question 1
**Q:** {q1}

**A:** {a1}

---

## Question 2
**Q:** {q2}

**A:** {a2}

---

## Question 3
**Q:** {q3}

**A:** {a3}

---

## Question 4
**Q:** {q4}

**A:** {a4}
"""

display(Markdown(qa_summary))

## Try Your Own Questions!

The conversation is still active. You can ask more questions by running the cell below.

In [ ]:
# Ask your own question!
my_question = "Can you explain the king - man + woman = queen example?"

my_answer = ask_question(my_question)

print(f"Q: {my_question}")
print(f"A: {my_answer}")

## Key Takeaways

In this notebook, you learned:

1. **Conversations API**: Use `conversations.create()` to maintain stateful chat sessions
2. **Context Management**: The API automatically remembers all previous messages, so follow-up questions work naturally
3. **Document Q&A**: By sending document content as initial context, you can build specialized assistants for any text

## When to Use Conversations API vs Responses API

**Use Conversations API when:**
- Building chat interfaces with multiple turns
- You need automatic context/history management
- Users will have back-and-forth conversations

**Use Responses API with `previous_response_id` when:**
- You want more control over context
- Building simple question-answer flows
- You need to manage context yourself

## Next Steps

Try:
- Loading different papers and comparing them
- Building a web interface around this Q&A system
- Adding citation extraction to show which part of the paper the answer comes from